In [1]:
import pandas as pd
import numpy as np
import sys

import pylab as plt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd

In [175]:
i = 1

def get_res_data(i):
    
    df_res_none = pd.read_csv(f'results/SE_results/results_{i}0.csv')
    df_res_low = pd.read_csv(f'results/SE_results/results_{i}3.csv')
    df_res_mid = pd.read_csv(f'results/SE_results/results_{i}2.csv')
    df_res_high = pd.read_csv(f'results/SE_results/results_{i}1.csv')
    
    return df_res_none, df_res_low, df_res_mid, df_res_high
    
def get_final_agent_data(i):

    df_fdata_none = pd.read_csv(f'results/SE_results/final_data_{i}0.csv').iloc[0:100]
    df_fdata_low = pd.read_csv(f'results/SE_results/final_data_{i}3.csv').iloc[0:100]
    df_fdata_mid = pd.read_csv(f'results/SE_results/final_data_{i}2.csv').iloc[0:100]
    df_fdata_high = pd.read_csv(f'results/SE_results/final_data_{i}1.csv').iloc[0:100]
    
    return df_fdata_none, df_fdata_low, df_fdata_mid, df_fdata_high

def get_initial_data(i):
    
    initial_df_comb = pd.read_csv(f'initial_data/initial_data_{i}.csv').iloc[0:100]
    
    return initial_df_comb

In [19]:
agent_states_cols = ['num_of_agents_received', 'num_of_agents_not_received', 'num_of_spreader_agents', 'num_of_disinterested_agents']

In [20]:
df_res_none, df_res_low, df_res_mid, df_res_high = get_res_data(i)

In [21]:
df_fdata_none, df_fdata_low, df_fdata_mid, df_fdata_high = get_final_agent_data(i)

In [41]:
def get_state_dist(attr):
    res = []

    for i in range(1,9):

        df_res_none, df_res_low, df_res_mid, df_res_high = get_res_data(i)
        res.append([df_res_none[attr].mean(), df_res_low[attr].mean(), df_res_mid[attr].mean(), df_res_high[attr].mean()])

    res = np.array(res)
    return list(res.mean(axis = 0))

In [43]:
recieved_agents = get_state_dist('num_of_agents_received')
not_received_agents = get_state_dist('num_of_agents_not_received')
spreader_agents = get_state_dist('num_of_spreader_agents')
disinterested_agents = get_state_dist('num_of_disinterested_agents')

In [209]:
index_names = ['received', 'notReceived', 'spreader', 'disinterested']
df_agent_states = pd.DataFrame([recieved_agents, not_received_agents, spreader_agents, disinterested_agents], columns = ['None', 'Low', 'Mid', 'High'])

In [210]:
df_agent_states.index = index_names

In [211]:
df_agent_states = df_agent_states.T
df_agent_states = df_agent_states.applymap(lambda x: f"{x:.3f}" if isinstance(x, float) else x)
df_agent_states

,received,notReceived,spreader,disinterested
None,90.127,9.873,11.257,66.948
Low,86.463,13.537,11.596,64.264
Mid,65.823,34.177,5.531,48.416
High,11.141,88.859,0.774,7.421


In [212]:
df_agent_states.to_csv('results/table/agent_state_dist.csv')

In [213]:
def get_active_users_metrics(i, thres_h, thres_l):

    df_fdata_none, df_fdata_low, df_fdata_mid, df_fdata_high = get_final_agent_data(i)
    df_fdata_initial = get_initial_data(i)
    
    act_users = [df_fdata_initial[df_fdata_initial['activity'] > thres_h].shape[0], 
                df_fdata_none[df_fdata_none['activity'] > thres_h].shape[0], 
                 df_fdata_low[df_fdata_low['activity'] > thres_h].shape[0], 
                 df_fdata_mid[df_fdata_mid['activity'] > thres_h].shape[0],
                 df_fdata_high[df_fdata_high['activity'] > thres_h].shape[0]]
    
    mid_act_users = [df_fdata_initial[((df_fdata_initial['activity'] >= thres_l) & (df_fdata_initial['activity'] <= thres_h))].shape[0],
                 df_fdata_none[((df_fdata_none['activity'] >= thres_l) & (df_fdata_none['activity'] <= thres_h))].shape[0], 
                 df_fdata_low[((df_fdata_low['activity'] >= thres_l) & (df_fdata_low['activity'] <= thres_h))].shape[0], 
                 df_fdata_mid[((df_fdata_mid['activity'] >= thres_l) & (df_fdata_mid['activity'] <= thres_h))].shape[0],
                 df_fdata_high[((df_fdata_high['activity'] >= thres_l) & (df_fdata_high['activity'] <= thres_h))].shape[0]]
            
    non_act_users = [df_fdata_initial[df_fdata_initial['activity'] < thres_l].shape[0],
                 df_fdata_none[df_fdata_none['activity'] < thres_l].shape[0], 
                 df_fdata_low[df_fdata_low['activity'] < thres_l].shape[0], 
                 df_fdata_mid[df_fdata_mid['activity'] < thres_l].shape[0],
                 df_fdata_high[df_fdata_high['activity'] < thres_l].shape[0]]
    
    return act_users, mid_act_users, non_act_users

In [214]:
res = []

for i in range(1,9):

    act_users, mid_act_users, non_act_users = get_active_users_metrics(i, 0.8, 0.2)
    res.append([act_users, mid_act_users, non_act_users])
    
res = np.array(res)
df_active_users = pd.DataFrame(res.mean(axis = 0), columns = ['initial', 'none', 'low', 'mid', 'high'])
df_active_users.index = ['active', 'med', 'nonActive']
df_active_users

,initial,none,low,mid,high
active,2.125,31.125,63.000,50.25,47.875
med,52.875,32.875,6.375,18.75,25.000
nonActive,45.000,36.000,30.625,31.00,27.125


In [215]:
act_users, mid_act_users, non_act_users

([1, 60, 72, 75, 40], [56, 1, 2, 3, 30], [43, 39, 26, 22, 30])

In [216]:
df_fdata_none.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'topic_1', 'topic_2', 'topic_3',
       'topic_4', 'topic_5', 'topic_6', 'privacy', 'activity', 'satisfaction',
       'lat_acc', 'lat_rej', 'id', 'pol_inclination'],
      dtype='object')

In [217]:
def get_polarized_users_metrics(i, thres):

    df_fdata_none, df_fdata_low, df_fdata_mid, df_fdata_high = get_final_agent_data(i)
    df_fdata_initial = get_initial_data(i)
    
    unpol_users = [df_fdata_initial[((df_fdata_initial['pol_inclination'] >= - thres) & (df_fdata_initial['pol_inclination'] <= thres))].shape[0],
                 df_fdata_none[((df_fdata_none['pol_inclination'] >= - thres) & (df_fdata_none['pol_inclination'] <= thres))].shape[0], 
                 df_fdata_low[((df_fdata_low['pol_inclination'] >= - thres) & (df_fdata_low['pol_inclination'] <= thres))].shape[0], 
                 df_fdata_mid[((df_fdata_mid['pol_inclination'] >= - thres) & (df_fdata_mid['pol_inclination'] <= thres))].shape[0],
                 df_fdata_high[((df_fdata_high['pol_inclination'] >= - thres) & (df_fdata_high['pol_inclination'] <= thres))].shape[0]]
            
    pol_users = [df_fdata_initial[((df_fdata_initial['pol_inclination'] < - thres) | (df_fdata_initial['pol_inclination'] > thres))].shape[0], 
                 df_fdata_none[((df_fdata_none['pol_inclination'] < - thres) | (df_fdata_none['pol_inclination'] > thres))].shape[0], 
                 df_fdata_low[((df_fdata_low['pol_inclination'] < - thres) | (df_fdata_low['pol_inclination'] > thres))].shape[0], 
                 df_fdata_mid[((df_fdata_mid['pol_inclination'] < - thres) | (df_fdata_mid['pol_inclination'] > thres))].shape[0],
                 df_fdata_high[((df_fdata_high['pol_inclination'] < - thres) | (df_fdata_high['pol_inclination'] > thres))].shape[0]]
    
    return unpol_users, pol_users

In [218]:
res = []

for i in range(1,9):

    unpol_users, pol_users = get_polarized_users_metrics(1, 0.5)
    res.append([unpol_users, pol_users])
    
res = np.array(res)
df_pol_users = pd.DataFrame(res.mean(axis = 0), columns = ['initial', 'none', 'low', 'mid', 'high'])
df_pol_users.index = ['pol', 'unPol']
df_pol_users

,initial,none,low,mid,high
pol,83.0,80.0,91.0,90.0,78.0
unPol,17.0,20.0,9.0,10.0,22.0


In [219]:
def get_sat_users_metrics(i):

    df_fdata_none, df_fdata_low, df_fdata_mid, df_fdata_high = get_final_agent_data(i)
    df_fdata_initial = get_initial_data(i)
    
    pos_sat_users = [df_fdata_initial[df_fdata_initial['satisfaction'] > 0].shape[0],
                    df_fdata_none[df_fdata_none['satisfaction'] > 0].shape[0],  df_fdata_low[df_fdata_low['satisfaction'] > 0].shape[0], 
                      df_fdata_mid[df_fdata_mid['satisfaction'] > 0].shape[0],  df_fdata_high[df_fdata_high['satisfaction'] > 0].shape[0]] 
    
    neg_sat_users = [df_fdata_initial[df_fdata_initial['satisfaction'] < 0].shape[0],
                    df_fdata_none[df_fdata_none['satisfaction'] < 0].shape[0],  df_fdata_low[df_fdata_low['satisfaction'] < 0].shape[0], 
                      df_fdata_mid[df_fdata_mid['satisfaction'] < 0].shape[0],  df_fdata_high[df_fdata_high['satisfaction'] < 0].shape[0]]
    
    zero_sat_users = [df_fdata_initial[df_fdata_initial['satisfaction'] == 0].shape[0],
                    df_fdata_none[df_fdata_none['satisfaction'] == 0].shape[0],  df_fdata_low[df_fdata_low['satisfaction'] == 0].shape[0], 
                      df_fdata_mid[df_fdata_mid['satisfaction'] == 0].shape[0],  df_fdata_high[df_fdata_high['satisfaction'] == 0].shape[0]]
    
    return pos_sat_users, neg_sat_users, zero_sat_users

In [220]:
res = []

for i in range(1,9):

    pos_sat_users, neg_sat_users, zero_sat_users = get_sat_users_metrics(1)
    res.append([pos_sat_users, neg_sat_users, zero_sat_users])
    
res = np.array(res)
df_sat_users = pd.DataFrame(res.mean(axis = 0), columns = ['initial', 'none', 'low', 'mid', 'high'])
df_sat_users.index = ['posSatUsers', 'negSatUsers', 'zeroSatUsers']
df_sat_users

,initial,none,low,mid,high
posSatUsers,0.0,0.0,0.0,1.0,78.0
negSatUsers,0.0,100.0,100.0,99.0,21.0
zeroSatUsers,100.0,0.0,0.0,0.0,1.0


In [221]:
df_secondary_metrics = pd.concat((df_active_users, df_pol_users, df_sat_users), axis = 0)
# df_secondary_metrics.T.to_csv('results/table/sec_metrics.csv')

In [222]:
df_secondary_metrics = df_secondary_metrics.T
df_secondary_metrics

,active,med,nonActive,pol,unPol,posSatUsers,negSatUsers,zeroSatUsers
initial,2.125,52.875,45.000,83.0,17.0,0.0,0.0,100.0
none,31.125,32.875,36.000,80.0,20.0,0.0,100.0,0.0
low,63.000,6.375,30.625,91.0,9.0,0.0,100.0,0.0
mid,50.250,18.750,31.000,90.0,10.0,1.0,99.0,0.0
high,47.875,25.000,27.125,78.0,22.0,78.0,21.0,1.0


In [223]:
df_secondary_metrics = df_secondary_metrics.applymap(lambda x: f"{x:.3f}" if isinstance(x, float) else x)
df_secondary_metrics

,active,med,nonActive,pol,unPol,posSatUsers,negSatUsers,zeroSatUsers
initial,2.125,52.875,45.000,83.000,17.000,0.000,0.000,100.000
none,31.125,32.875,36.000,80.000,20.000,0.000,100.000,0.000
low,63.000,6.375,30.625,91.000,9.000,0.000,100.000,0.000
mid,50.250,18.750,31.000,90.000,10.000,1.000,99.000,0.000
high,47.875,25.000,27.125,78.000,22.000,78.000,21.000,1.000


In [224]:
df_secondary_metrics.to_csv("results/table/sec_metrics.csv")